In [1]:
import pandas as pd
import numpy as np
import qcew_2014_2021
import qcew_2010_2013
# import all functions from qcew module
from qcew_2014_2021 import *
from qcew_2010_2013 import *



In [2]:
FOSSIL_NAICS_CODES = ['2121','211','213','23712','486','4247','22112']
old_years = ['2010','2011','2012','2013']


2014-2021 data (via API)

In [3]:
#FOR FOSSIL

fossil_industry_data = get_2014_industry_data('fossil')

cleaned_fossil_data = get_cleaned_2014_industry_data(fossil_industry_data)

fossil_clean_with_geo = get_2014_industry_with_bls_geographic_tag(cleaned_fossil_data)

aggregated_fossil_data = aggregate_data_by_type(fossil_clean_with_geo,'fossil')



/Users/mariacastillo-coding/Documents/GitHub/rmi-energy-communities/src/energy_comms/extract/qcew_2014_2021.py:94: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  data['geographic_level'] = np.where(data['area_title'].str.contains('(Combined)'),'aggregated_stat_area',data['geographic_level'])


In [4]:
#FOR TOTALS

total_employment_data = get_2014_industry_data('total')

cleaned_total_data = get_cleaned_2014_industry_data(total_employment_data)

total_clean_with_geo = get_2014_industry_with_bls_geographic_tag(cleaned_total_data)

aggregated_total_data = aggregate_data_by_type(total_clean_with_geo,'total')

# drop ownership and industry column

aggregated_total_data = aggregated_total_data.drop(columns=['own_code','industry_code'])

/Users/mariacastillo-coding/Documents/GitHub/rmi-energy-communities/src/energy_comms/extract/qcew_2014_2021.py:94: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  data['geographic_level'] = np.where(data['area_title'].str.contains('(Combined)'),'aggregated_stat_area',data['geographic_level'])


In [5]:
# bring together

#fossil

total_fossil_data = aggregated_fossil_data.groupby(['area_fips','area_title','geographic_level','year']).agg({'annual_avg_emplvl':'sum'}).reset_index()

total_fossil_data = total_fossil_data.rename(columns={'annual_avg_emplvl':'fossil_employees'})

#aggregated total

aggregated_total_data = aggregated_total_data.rename(columns={'annual_avg_emplvl':'total_employees'})

In [6]:
# merge on year

combined_attempt = aggregated_total_data.merge(total_fossil_data,on=['area_fips','area_title','geographic_level','year'],how='left')

In [ ]:
#BONEYARD PROCESSING FOR THRESHOLDS

combined_attempt['percent_of_fossil_employment'] = combined_attempt['fossil_employees'] / combined_attempt ['total_employees'] * 100


# criteria only relevant for MSAs and (some counties)

fossil_employment_ratios = combined_attempt.query("geographic_level== 'metropolitan_stat_area' | geographic_level == 'county'")

fossil_employment_ratios[fossil_employment_ratios['percent_of_fossil_employment'] > 0.17]

2010-2013 data (via online downloading)

In [2]:
test = extract_2010()

In [3]:
test

,area_fips,own_code,industry_code,agglvl_code,size_code,year,qtr,disclosure_code,area_title,own_title,...,oty_total_annual_wages_chg,oty_total_annual_wages_pct_chg,oty_taxable_annual_wages_chg,oty_taxable_annual_wages_pct_chg,oty_annual_contributions_chg,oty_annual_contributions_pct_chg,oty_annual_avg_wkly_wage_chg,oty_annual_avg_wkly_wage_pct_chg,oty_avg_annual_pay_chg,oty_avg_annual_pay_pct_chg
0,37059,0,10,70,0,2010,A,NaN,"Davie County, North Carolina",Total Covered,...,-2277090,-0.8,-1662633,-1.3,-77964,-3.4,8,1.4,390,1.3
1,37059,1,10,71,0,2010,A,NaN,"Davie County, North Carolina",Federal Government,...,-30154,-1.0,0,0.0,0,0.0,-84,-10.0,-4377,-10.1
2,37059,1,102,72,0,2010,A,NaN,"Davie County, North Carolina",Federal Government,...,-30154,-1.0,0,0.0,0,0.0,-84,-10.0,-4377,-10.1
3,37059,1,1021,73,0,2010,A,NaN,"Davie County, North Carolina",Federal Government,...,-188194,-6.9,0,0.0,0,0.0,18,2.1,901,2.0
4,37059,1,1028,73,0,2010,A,NaN,"Davie County, North Carolina",Federal Government,...,158040,74.2,0,0.0,0,0.0,-235,-38.3,-12241,-38.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14284496,49053,5,999,75,0,2013,A,NaN,"Washington County, Utah",Private,...,38085,41.8,33148,77.8,523,94.6,-367,-36.6,-19090,-36.7
14284497,49053,5,9999,76,0,2013,A,NaN,"Washington County, Utah",Private,...,38085,41.8,33148,77.8,523,94.6,-367,-36.6,-19090,-36.7
14284498,49053,5,99999,77,0,2013,A,NaN,"Washington County, Utah",Private,...,38085,41.8,33148,77.8,523,94.6,-367,-36.6,-19090,-36.7
14284499,49053,5,999999,78,0,2013,A,NaN,"Washington County, Utah",Private,...,38085,41.8,33148,77.8,523,94.6,-367,-36.6,-19090,-36.7


In [9]:
for yr in old_years:
    download_qcew_zip_files(yr)

In [10]:
data_2010 = make_year_df(2010)
data_2011 = make_year_df(2011)
data_2012 = make_year_df(2012)
data_2013 = make_year_df(2013)

data_dfs = [data_2010,data_2011,data_2012,data_2013]

df = pd.concat(data_dfs,ignore_index=True)


In [11]:
df_geo = get_2014_industry_with_bls_geographic_tag(df)

/Users/mariacastillo-coding/Documents/GitHub/rmi-energy-communities/src/energy_comms/extract/qcew_2014_2021.py:94: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  data['geographic_level'] = np.where(data['area_title'].str.contains('(Combined)'),'aggregated_stat_area',data['geographic_level'])


In [12]:
#make total df of 2010-2013 data
total_employment = df_geo.query("industry_code == '10' & own_code == 0")
#rename employee column as total for future merge
total_employment = total_employment.rename(columns={'annual_avg_emplvl':'total_employees'})

#keep columns that are in run_qcew.ipynb
total_employment_df = total_employment[['area_fips','area_title','geographic_level','year','total_employees']]


In [13]:
#make fossil employment

FOSSIL_NAICS_CODES = ['2121','211','213','23712','486','4247','22112']


fossil_employment = df_geo.loc[df_geo['industry_code'].isin(FOSSIL_NAICS_CODES)]


# group by all ownership codes

total_fossil_employment = fossil_employment.groupby(['area_fips','area_title','geographic_level','year']).agg({'annual_avg_emplvl':'sum'}).reset_index()

total_fossil_employment = total_fossil_employment.rename(columns={'annual_avg_emplvl':'fossil_employees'})

total_fossil_employment_short = total_fossil_employment[['area_fips','area_title','geographic_level','year','fossil_employees']]


In [14]:
#combine for total and fossil 2010-2013

combined_2010= total_employment_df.merge(total_fossil_employment_short,on=['area_fips','area_title','geographic_level','year'],how='left')
combined_2010['year'] = combined_2010['year'].astype(str)

In [22]:
#group all

combined_dfs = [combined_2010,combined_attempt]

combined = pd.concat(combined_dfs,ignore_index=True)

combined

,area_fips,area_title,geographic_level,year,total_employees,fossil_employees
0,37059,"Davie County, North Carolina",county,2010,9479,0.0
1,49045,"Tooele County, Utah",county,2010,15628,0.0
2,17045,"Edgar County, Illinois",county,2010,6273,0.0
3,45085,"Sumter County, South Carolina",county,2010,34670,8.0
4,55073,"Marathon County, Wisconsin",county,2010,64645,16.0
...,...,...,...,...,...,...
168747,USNMS,U.S. Nonmetropolitan Area Counties (combined),U.S. Nonmetropolitan Area Counties (combined),2017,19449252,NaN
168748,USNMS,U.S. Nonmetropolitan Area Counties (combined),U.S. Nonmetropolitan Area Counties (combined),2018,19662669,NaN
168749,USNMS,U.S. Nonmetropolitan Area Counties (combined),U.S. Nonmetropolitan Area Counties (combined),2019,19771027,NaN
168750,USNMS,U.S. Nonmetropolitan Area Counties (combined),U.S. Nonmetropolitan Area Counties (combined),2020,18904204,NaN
